In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.algorithms.ubc_exploitability import game_tree_size

from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_clusters import projectPCA, projectUMAP, fitGMM, projectTSNE
from open_spiel.python.examples.ubc_sample_game_tree import NodeType
from open_spiel.python.examples.ubc_evaluate_policy import run_eval

from auctions.webutils import *


from open_spiel.python.algorithms.random_agent import RandomAgent
import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from bokeh.palettes import Category10_10, Magma256, Spectral10, Category20_20
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from collections import defaultdict
output_notebook()

Loading BokehJS ...

In [2]:
# GAME_NAME = 'sats_5.json'
# game = smart_load_clock_auction(GAME_NAME)
# game_config = game_config = load_game_config(GAME_NAME)
# config = read_config('april25/mlp')
# config['num_training_episodes'] = 100
# env_and_model = setup(game, game_config, config)
# # for i in range(game.num_players()):
# #     env_and_model.agents[i] = RandomAgent(i, game.num_distinct_actions())


In [3]:
# OPTIONS
EXPERIMENT_NAME = 'sats_games_5'
RUN_NAME = 'sats_5-april26mlp-100'
### Load checkpoint
checkpoint = get_checkpoint_by_name(EXPERIMENT_NAME, RUN_NAME)
game = load_game(checkpoint.game)
checkpoint.equilibrium_solver_run.config['device'] = 'cpu'
env_and_model = db_checkpoint_loader(checkpoint)
# for i in range(game.num_players()):
#     env_and_model.agents[i] = RandomAgent(i, game.num_distinct_actions())


Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/sats_5.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/sats_5.json
Done config parsing


In [57]:
def prof():
    return run_eval(env_and_model, 10_000)

In [58]:
import cProfile, pstats
profiler = cProfile.Profile()
profiler.enable()
eval_output = prof()
profiler.disable()


In [59]:
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()


         95394815 function calls (93799342 primitive calls) in 308.168 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  308.549  154.274 /home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3215(run_code)
        2    0.000    0.000  308.549  154.274 {built-in method builtins.exec}
        1    0.000    0.000  308.549  308.549 /tmp/ipykernel_8551/3371018378.py:4(<module>)
        1    0.119    0.119  308.549  308.549 /tmp/ipykernel_8551/1006448451.py:1(prof)
        1    6.605    6.605  308.429  308.429 /apps/open_spiel/open_spiel/python/examples/ubc_evaluate_policy.py:36(run_eval)
   653208    4.955    0.000  256.673    0.000 /apps/open_spiel/open_spiel/python/examples/ubc_decorators.py:49(step)
   618895    4.831    0.000  225.844    0.000 /apps/open_spiel/open_spiel/python/pytorch/ubc_nfsp.py:208(step)
   588895   43.875    0.000  219.145 

In [49]:
%%time
run_eval(env_and_model, 1000)

CPU times: user 3min 1s, sys: 336 ms, total: 3min 2s
Wall time: 23.5 s


{'walltime': 23.10131812095642,
 'rewards': defaultdict(list,
             {0: [267.2076599803692,
               455.7655506096775,
               280.87293703446176,
               -844.262832336974,
               430.6540597124997,
               -938.829942388755,
               496.5639235151423,
               -876.0678638100655,
               -1033.7299284787086,
               -568.7040203233485,
               -804.2512437507303,
               -319.40286206467545,
               -461.1095189010002,
               -596.4669716895219,
               588.8290530040716,
               -328.25637189605476,
               236.5841463223594,
               322.2661947115821,
               299.1747762661616,
               -318.9323676100714,
               -529.3144136421886,
               657.9040572549998,
               -417.0008915533986,
               -713.9057499962418,
               -787.9567530366626,
               484.8554778809998,
               382.9415894994011,


In [47]:
import pprofile
profiler = pprofile.Profile()
with profiler:
    run_eval(env_and_model, 1000)




In [48]:
# profiler.print_stats()
profiler.dump_stats("greg/profile.txt")


In [12]:
from open_spiel.python.examples.ubc_utils import fast_choice

In [16]:
p = np.random.random(512)
p = p / p.sum()

options = np.arange(len(p))

In [17]:
%%timeit 
fast_choice(options, p)

71.1 µs ± 2.9 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
np.searchsorted(

<function numpy.searchsorted(a, v, side='left', sorter=None)>

In [21]:
np.cumsum(p)

In [42]:
def faster_choice(options, p):
    x = np.random.rand()
    cdf = np.cumsum(p)
    x = np.random.rand() * cdf[-1] # normalize to avoid rounding issues
    i = np.searchsorted(cdf, x)
    return options[i]


In [43]:
%%timeit
faster_choice(options, p)

11.8 µs ± 201 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [39]:
import pprofile
profiler = pprofile.Profile()
with profiler:
    for i in range(10000):
        faster_choice(options, p)




In [41]:
profiler.print_stats()

Total duration: 1.01676s
File: /home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py
File duration: 0.429763s (42.27%)
Line #|      Hits|         Time| Time per hit|      %|Source code
------+----------+-------------+-------------+-------+-----------
     1|         0|            0|            0|  0.00%|"""Module containing non-deprecated functions borrowed from Numeric.
     2|         0|            0|            0|  0.00%|
     3|         0|            0|            0|  0.00%|"""
     4|         0|            0|            0|  0.00%|import functools
     5|         0|            0|            0|  0.00%|import types
     6|         0|            0|            0|  0.00%|import warnings
     7|         0|            0|            0|  0.00%|
     8|         0|            0|            0|  0.00%|import numpy as np
     9|         0|            0|            0|  0.00%|from . import multiarray as mu
    10|         0|            0|            0| 

In [1]:
from __future__ import print_function
from math import sin, cos, radians
import timeit
def bench():
    product = 1.0
    for counter in range(1, 1000, 1):
        for dex in list(range(1, 360, 1)):
            angle = radians(dex)
            product *= sin(angle)**2 + cos(angle)**2
    return product


In [2]:
%%timeit
bench()



123 ms ± 507 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
